In [0]:
%sql
-- 1. Create the main Catalog for the project
CREATE CATALOG IF NOT EXISTS agriculture;

-- 2. Switch to this catalog
USE CATALOG agriculture;

-- 3. Create the 3 Layers (Schemas)
CREATE SCHEMA IF NOT EXISTS bronze
COMMENT 'Raw data ingestion layer';

CREATE SCHEMA IF NOT EXISTS silver
COMMENT 'Cleaned, joined, and enriched data';

CREATE SCHEMA IF NOT EXISTS gold
COMMENT 'Business-level aggregates for the recommendation app';

-- 4. Create a Volume for your raw CSV uploads
CREATE VOLUME IF NOT EXISTS bronze.raw_uploads;

In [0]:
%pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 111.2 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
import os

os.environ["KAGGLE_USERNAME"] = "manjunathask"
os.environ["KAGGLE_KEY"] = "KGAT_d728f5afde6fded6eee89ecf9a4e2593"

print("Kaggle credentials configured!")

Kaggle credentials configured!


In [0]:
%sh
cd /Volumes/agriculture/bronze/raw_uploads
kaggle datasets download -d aksahaha/crop-recommendation
ls -lh

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:2557: UserWarning: Spark Connect Session expired on the server. Please generate a new session by detaching and reattaching the compute if in a Databricks notebook or job or by calling DatabricksSession.builder.getOrCreate() if using Databricks Connect.
  warnings.warn(


Dataset URL: https://www.kaggle.com/datasets/aksahaha/crop-recommendation
License(s): copyright-authors


100%|██████████| 62.6k/62.6k [00:00<00:00, 316kB/s]



total 63K
-rwxrwxrwx 1 spark-e6a421e1-2908-4f2a-b79c-7a nogroup 63K Jan 26  2026 crop-recommendation.zip


In [0]:
%sh
cd /Volumes/agriculture/bronze/raw_uploads
unzip -o crop-recommendation.zip
rm -f crop-recommendation.zip

Archive:  crop-recommendation.zip
  inflating: Crop_recommendation.csv  
total 146K
-rwxrwxrwx 1 spark-e6a421e1-2908-4f2a-b79c-7a nogroup 146K Jan 26 14:58 Crop_recommendation.csv


In [0]:
%sh
cd /Volumes/agriculture/bronze/raw_uploads
kaggle datasets download -d anshumish/crop-yield-data-with-soil-and-weather-dataset
unzip -o crop-yield-data-with-soil-and-weather-dataset.zip
rm -f crop-yield-data-with-soil-and-weather-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/anshumish/crop-yield-data-with-soil-and-weather-dataset
License(s): CC0-1.0


100%|██████████| 457k/457k [00:00<00:00, 2.74MB/s]



Archive:  crop-yield-data-with-soil-and-weather-dataset.zip
  inflating: crop_yield.csv          
  inflating: state_soil_data.csv     
  inflating: state_weather_data_1997_2020.csv  


In [0]:
%sh
cd /Volumes/agriculture/bronze/raw_uploads
kaggle datasets download -d vandeetshah/india-commodity-wise-mandi-dataset

mkdir -p mandi_data_temp
unzip -o india-commodity-wise-mandi-dataset.zip -d mandi_data_temp

mkdir -p mandi_data
files=(
    "Beaten Rice.csv"
    "Maize.csv"
    "Kabuli Chana(Chickpeas-White).csv"
    "Lentil (Masur)(Whole).csv"
    "Pomegranate.csv"
    "Banana - Green.csv"
    "Mango.csv"
    "Grapes.csv"
    "Pineapple.csv"
    "Orange.csv"
    "Papaya.csv"
    "Tender Coconut.csv"
    "Cotton.csv"
    "Jute.csv"
    "Coffee.csv"
)

for file in "${files[@]}"; do
    cp "mandi_data_temp/$file" mandi_data/ 2>/dev/null && echo " Copied: $file" || echo " Not found: $file"
done

rm -rf mandi_data_temp
rm -f india-commodity-wise-mandi-dataset.zip

echo "Done! Files in mandi_data:"
ls -lh mandi_data/

Dataset URL: https://www.kaggle.com/datasets/vandeetshah/india-commodity-wise-mandi-dataset
License(s): apache-2.0


100%|██████████| 404M/404M [00:07<00:00, 56.8MB/s]



Archive:  india-commodity-wise-mandi-dataset.zip
  inflating: mandi_data_temp/Ajwan.csv  
  inflating: mandi_data_temp/Alasande Gram.csv  
  inflating: mandi_data_temp/Almond(Badam).csv  
  inflating: mandi_data_temp/Alsandikai.csv  
  inflating: mandi_data_temp/Amaranthus.csv  
  inflating: mandi_data_temp/Ambada Seed.csv  
  inflating: mandi_data_temp/Amla(Nelli Kai).csv  
  inflating: mandi_data_temp/Amphophalus.csv  
  inflating: mandi_data_temp/Antawala.csv  
  inflating: mandi_data_temp/Anthorium.csv  
  inflating: mandi_data_temp/Apple.csv  
  inflating: mandi_data_temp/Apricot(Jardalu-Khumani).csv  
  inflating: mandi_data_temp/Arecanut(Betelnut-Supari).csv  
  inflating: mandi_data_temp/Arhar (Tur-Red Gram)(Whole).csv  
  inflating: mandi_data_temp/Arhar Dal(Tur Dal).csv  
  inflating: mandi_data_temp/Ashgourd.csv  
  inflating: mandi_data_temp/Astera.csv  
  inflating: mandi_data_temp/Avare Dal.csv  
  inflating: mandi_data_temp/BOP.csv  
  inflating: mandi_data_temp/Bajra(P

In [0]:
%sh
cd /Volumes/agriculture/bronze/raw_uploads/mandi_data
ls 

Banana - Green.csv
Beaten Rice.csv
Coffee.csv
Cotton.csv
Grapes.csv
Jute.csv
Kabuli Chana(Chickpeas-White).csv
Lentil (Masur)(Whole).csv
Maize.csv
Mango.csv
Orange.csv
Papaya.csv
Pineapple.csv
Pomegranate.csv
Tender Coconut.csv


In [0]:
%sh
cd /Volumes/agriculture/bronze/raw_uploads
ls 

Crop_recommendation.csv
crop_yield.csv
mandi_data
state_soil_data.csv
state_weather_data_1997_2020.csv


In [0]:
%sh
cd /Volumes/agriculture/bronze/raw_uploads
kaggle datasets download -d aravindpcoder/india-crop-production-state-wise

Dataset URL: https://www.kaggle.com/datasets/aravindpcoder/india-crop-production-state-wise
License(s): CC0-1.0


100%|██████████| 1.96M/1.96M [00:01<00:00, 1.90MB/s]


In [0]:
%sh
cd /Volumes/agriculture/bronze/raw_uploads
unzip -o india-crop-production-state-wise.zip
rm -f india-crop-production-state-wise.zip
ls -lh

Archive:  india-crop-production-state-wise.zip
  inflating: crop_production.csv     
total 17M
-rwxrwxrwx 1 spark-c650b1bb-25e2-4d82-82a9-84 nogroup  15M Jan 28  2026 crop_production.csv
-rwxrwxrwx 1 nobody                           nogroup 146K Jan 26 14:58 Crop_recommendation.csv
-rwxrwxrwx 1 nobody                           nogroup 1.5M Jan 26 15:00 crop_yield.csv
drwxrwxrwx 2 nobody                           nogroup 4.0K Jan 28 22:08 mandi_data
-rwxrwxrwx 1 nobody                           nogroup  741 Jan 26 15:00 state_soil_data.csv
-rwxrwxrwx 1 nobody                           nogroup  26K Jan 26 15:00 state_weather_data_1997_2020.csv


In [0]:
%%sh
cd /Volumes/agriculture/bronze/raw_uploads
ls

crop_production.csv
Crop_recommendation.csv
crop_yield.csv
mandi_data
state_soil_data.csv
state_weather_data_1997_2020.csv
